In [12]:
from datetime import datetime

from db_comm import PortfolioDBManager, DATABASE_NAME

In [13]:
pm = PortfolioDBManager(DATABASE_NAME)

Calculation Motor

In [14]:
from portfolio_test.motor import CalculationMotor

In [15]:
cm = CalculationMotor("AAPL")

In [16]:
pm.sell_all_assets()
starting_off_cash = 100
# withdraw cash
pm.record_transaction(
    tx_type="WITHDRAW",  # WITHDRAW or DEPOSIT
    ticker="CASH",
    shares=pm.get_cash_balance(),
    actual_price=1,
    tx_datetime=datetime.now(),
)
pm.record_transaction(
    tx_type="DEPOSIT",  # WITHDRAW or DEPOSIT
    ticker="CASH",
    shares=starting_off_cash,
    actual_price=1,
    tx_datetime=datetime.now(),
)

   -> Cash balance adjusted by 209.9517
2026-01-05 21:40:55: ✅ Recorded SELL: AAPL @ 209.9517. Snapshot updated.
2026-01-05 21:40:55: ✅ Recorded WITHDRAW: CASH @ 209.9517. Snapshot updated.
2026-01-05 21:40:55: ✅ Recorded DEPOSIT: CASH @ 100.0. Snapshot updated.


Build calculation motor for AAPL

In [17]:
data = cm.df
data

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2020-01-02,688.757977,698.895014,686.316756,698.313764,673.954834,135480400,0.0,0.0
2020-01-03,690.873736,698.848469,689.362500,691.524733,667.402503,146322800,0.0,0.0
2020-01-06,683.061770,697.406980,680.643750,697.034972,672.720600,118387200,0.0,0.0
2020-01-07,697.127991,699.592486,691.641026,693.756784,669.556931,108872000,0.0,0.0
2020-01-08,690.897009,707.823006,690.897009,704.916756,680.327511,132079200,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-12-26,2549.688034,2560.940955,2537.597864,2542.619943,2542.619943,21521800,0.0,0.0
2025-12-29,2536.017023,2551.547864,2532.855057,2545.968091,2545.968091,23715200,0.0,0.0
2025-12-30,2537.132977,2548.943875,2532.203989,2539.643875,2539.643875,22139600,0.0,0.0


In [18]:
import numpy as np

for date in data.index:
    print(f"- - - - - Date: {date.date()} - - - - -")
    # if price today is lower than yesterday, buy as much as possible
    data['Adj_Close_ydy'] = data['Adj Close'].shift(1)
    if date != data.index.min():
        price = data.loc[date, "Adj Close"]  # price in SEK
        price_yesterday = data.loc[date, "Adj_Close_ydy"]

        if price < price_yesterday:
            #print("Price dropped, buying AAPL")
            cash_balance = pm.get_cash_balance()
            shares_to_buy = np.divide(cash_balance, price)
            if shares_to_buy > 0:
                print("Buying AAPL")
                pm.record_transaction(
                    tx_type="BUY",
                    ticker="AAPL",
                    shares=shares_to_buy,
                    actual_price=price,
                    tx_datetime=date,
                    currency="SEK",
                )
        # if price today is higher than yesterday, sell all shares
        elif price > price_yesterday:
            portfolio = pm.get_portfolio_snapshot()
            if "AAPL" in list(portfolio["ticker"]):
                shares_to_sell = portfolio[portfolio["ticker"] == "AAPL"]["net_shares"].iloc[0]
                if shares_to_sell > 0:
                    pm.record_transaction(
                        tx_type="SELL",
                        ticker="AAPL",
                        shares=shares_to_sell,
                        actual_price=price,
                        tx_datetime=date,
                        currency="SEK",
                    )


- - - - - Date: 2020-01-02 - - - - -
- - - - - Date: 2020-01-03 - - - - -
Buying AAPL
   -> Cash balance adjusted by -100.0
2020-01-03 00:00:00: ✅ Recorded BUY: AAPL @ 100.0. Snapshot updated.
- - - - - Date: 2020-01-06 - - - - -
   -> Cash balance adjusted by 100.7968
2020-01-06 00:00:00: ✅ Recorded SELL: AAPL @ 100.7968. Snapshot updated.
- - - - - Date: 2020-01-07 - - - - -
Buying AAPL
   -> Cash balance adjusted by -100.7968
2020-01-07 00:00:00: ✅ Recorded BUY: AAPL @ 100.7968. Snapshot updated.
- - - - - Date: 2020-01-08 - - - - -
   -> Cash balance adjusted by 102.4182
2020-01-08 00:00:00: ✅ Recorded SELL: AAPL @ 102.4182. Snapshot updated.
- - - - - Date: 2020-01-09 - - - - -
- - - - - Date: 2020-01-10 - - - - -
- - - - - Date: 2020-01-13 - - - - -
- - - - - Date: 2020-01-14 - - - - -
Buying AAPL
   -> Cash balance adjusted by -102.4182
2020-01-14 00:00:00: ✅ Recorded BUY: AAPL @ 102.4182. Snapshot updated.
- - - - - Date: 2020-01-15 - - - - -
- - - - - Date: 2020-01-16 - - - - 

In [19]:
pm.get_portfolio_snapshot()

,ticker,net_shares,last_trade_price,total_position_value
0,AAPL,0.129995,2539.643875,330.1408


Get value of portfolio

In [20]:
last_price = data.loc[data.index.max(), "Adj Close"]
# get shares held in portfolio times last price to get value
portfolio = pm.get_portfolio_snapshot()
aapl_shares = 0
if "AAPL" in list(portfolio["ticker"]):
    aapl_shares = portfolio[portfolio["ticker"] == "AAPL"]["net_shares"].iloc[0]
aapl_value = aapl_shares * last_price
cash_value = pm.get_cash_balance()
total_value_portfolio = aapl_value + cash_value
print(f"Total portfolio value: {total_value_portfolio} SEK")

Total portfolio value: 327.63829585275846 SEK


In [21]:
# find Price_SEK at max date
first_price = data.loc[data.index.min(), "Adj Close"]
earnings = pm.get_portfolio_snapshot()["total_position_value"].sum()
holding_gain = last_price / first_price - 1
trading_gain = earnings / starting_off_cash - 1

print(f"Last price: {(holding_gain) * 100:.2f} %, Earnings: {(trading_gain) * 100:.2f} %")
if trading_gain < holding_gain:
    print("Overall worse than buy and hold. "
          f"Specifically, just buying would have given you {((holding_gain) / (trading_gain)):.2f} times better results.")
else:
    print(f"Overall better than buy and hold")
    if holding_gain > 0:
        print(f"{((trading_gain) / (holding_gain)):.2f} times better than buy and hold")
    elif trading_gain > 0:
        print("Holding was a loss, but trading made a gain")
    else:
        print("Both trading and holding were a loss but trading lost less")

Last price: 273.97 %, Earnings: 230.14 %
Overall worse than buy and hold. Specifically, just buying would have given you 1.19 times better results.
